<a href="https://colab.research.google.com/github/mvarentsov/UClim4HSE-2026/blob/main/scripts/Lect02_Colab_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Монтируем гугл-диск к виртуальной машине

In [ ]:
import sys, os
from google.colab import drive
drive.mount('/content/drive')

## Скачиваем репозиторий *курса*

In [ ]:


%cd /content/drive/MyDrive/
%pwd


os.makedirs ('UClim4HSE', exist_ok = True)

%cd /content/drive/MyDrive/UClim4HSE/
%pwd

if not os.path.isdir('/content/drive/MyDrive/UClim4HSE/UClim4HSE-2026'):
  print ('cloning repository')
  !git clone https://github.com/mvarentsov/UClim4HSE-2026.git
else:
  print ('updating repository')
  %cd /content/drive/MyDrive/UClim4HSE/UClim4HSE-2026/
  ! git pull

%cd /content/drive/MyDrive/UClim4HSE/


In [ ]:
sys.path.append('/content/drive/MyDrive/UClim4HSE/UClim4HSE-2026/scripts/')
import weaclim_utils
from datetime import date, datetime, timedelta

In [ ]:

station_id = 27518     # идентификатор станции, с которой нужны наблюдения
start = date (2023,1,1) # начало периода загрузки данных
end = date (2024,1,1) #date.today()      # конец  периода загрузки данных

weaclim_utils.load_weaclim4period (station_id, start, end, 'weaclim_data')


In [ ]:
df_urb = weaclim_utils.read_weaclim_dir ('weaclim_data/27605')
df_rur = weaclim_utils.read_weaclim_dir ('weaclim_data/27518')

df_rur.head()

In [ ]:
df_urb = df_urb.rename(columns = {x:'urb_' + x for x in df_urb.columns})
df_rur = df_rur.rename(columns = {x:'rur_' + x for x in df_rur.columns})

df = df_urb.merge (df_rur, left_index=True, right_index=True)
display(df.head())


In [ ]:
plt.plot(df_rur.index)

In [ ]:
from matplotlib import pyplot as plt

%matplotlib inline
plt.rcParams['figure.dpi'] = 200

# plt.figure()
# df['rur_t2m'].plot(color = 'blue')
# df['urb_t2m'].plot(color = 'red')
# plt.ylabel ('Temperature[K]')

# df_sel = df['2023-08':'2023-09']
# plt.figure()
# df_sel['rur_t2m'].plot(color = 'blue')
# df_sel['urb_t2m'].plot(color = 'red')
# plt.ylabel ('Temperature[K]')

#df_sel = df['2023-08':'2023-09-01']
plt.figure()
df['rur_t2m'].plot(color = 'blue')
df['urb_t2m'].plot(color = 'red')
plt.ylabel ('Temperature[K]')
#plt.xlim((datetime(2023,8,1), datetime(2023,9,1)))



## Интерактивный график с помощью plotly

In [ ]:
import pandas as pd
pd.options.plotting.backend = "plotly"

fig = df.sort_index()[['rur_t2m', 'urb_t2m']].plot(color_discrete_map={'rur_t2m':'blue', 'urb_t2m':'red'})
fig.show()


In [ ]:
#df_day_mean = df.resample ('1ME').mean()
df_day_mean = df.groupby(df.index.month).min()


display(df_day_mean)

fig = px.line(df_day_mean, y=['rur_t2m', 'urb_t2m'], title='Simple Time Series Plot') #, labels={'Value': 'Cumulative Value'})
fig.data[0].line.color = 'blue'  # Color for 'rur_t2m'
fig.data[1].line.color = 'orange'  # Color for 'urb_t2m'
fig.show()
